In [2]:
import random

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10,
         'Queen':10, 'King':10, 'Ace':11}

playing = True

class Card():
    '''
    class for creat card and print it
    attributes: suit and rank
    '''
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        
    def __str__(self):
        return self.rank + ' of ' + self.suit
    
class Deck():
    '''
    deck of cards
    usually it use 4 dekcs
    '''
    def __init__(self, count = 1):
        self.deck = []
        self.deck = [Card(suit, rank) for suit in suits for rank in ranks] * count
        

    def __str__(self):
        deck_comp = ''
        for card in self.deck:
            deck_comp += '\n' + card.__str__()
        return 'The deck has: ' + deck_comp   
    
    def shufle(self):
        random.shuffle(self.deck)
        
    def deal(self):
        '''
        take a card from deck
        '''
        return self.deck.pop()
    
    def check_deck(self):
        '''
        if deck is almost empty the end of the game
        '''
        return len(self.deck) < 15
    
    
class Hand():
    '''
    class for player 
    '''
    def __init__(self):
        self.cards = []
        self.value = 0
         
    def add_card(self, card):
        '''
        take a card from deck in hand
        if it is Ace and in the hand is more than 10 points then add 1 point otherwise 11
        '''
        self.cards.append(card)
        if card.rank == 'Ace' and self.value > 10:
            self.value += 1
        else:
            self.value += values[card.rank]
      
              
    def clear(self):
        '''
        clean hand before new game
        '''
        self.cards = []
        self.value = 0
        
    
class Chips:
    '''
    cheaps for player
    default 100 
    '''
    def __init__(self, total = 100):
        self.total = total  # This can be set to a default value or supplied by a user input
        self.bet = 0
        
    def win_bet(self):
        self.total += self.bet
    
    def lose_bet(self):
        self.total -= self.bet
        
        
        
def take_bet(chips):
    '''
    take a bet from player
    check the bet is correct and less then total 
    '''
    while True:
        try:
            chips.bet = int(input('Take a bet, please: '))
        except ValueError:
            print('Sorry, a bet must be an integer!')
        else:
            if chips.bet > chips.total:
                print('Sorty! You take wrong bet, too much', chips.total)
            else:
                print('The bet is OK')
                break

            
            
def hit(deck,hand):
    '''
    take first card form deck and add in hand 
    '''
    hand.add_card(deck.deal())
     
    
def hit_or_stand(deck,hand):
    '''
    hit card in hand or stand
    playing - false for out of loop
    '''
    global playing  # to control an upcoming while loop
    state = ''
    
    while state not in ['s', 'h']:
        try:
            state = input('Would you like stand or hit? Enter s or h').lower()
        except:
            print("Sorry, please try again.")
            continue
    if state == 'h':
        hit(deck, hand)
    else:
        print('Player stands. Dealer is playing.')
        playing = False
        
            
            
            
def show_some(player,dealer):
    '''
    show hand player and one card dealer hand
    '''
    print("\nDealer's Hand:")
    print(" <card hidden>")
    print('',dealer.cards[1]) 
    
    print("\nPlayer's Hand:", *player.cards, sep='\n ')
    print("Player's Hand =", player.value)
    
    
    
def show_all(player,dealer):
    '''
    show all cards in hands
    '''
    print("\nDealer's Hand:", *dealer.cards, sep='\n ')
    print("Dealer's Hand =",dealer.value)
    print("\nPlayer's Hand:", *player.cards, sep='\n ')
    print("Player's Hand =",player.value)
    
    
def player_busts(player,dealer):
    if player.value > 21:
        player.chips.lose_bet()
        print('The player is BUST')
        return True
    return False

def player_wins(player,dealer):
    if player.value == 21:
        player.chips.win_bet()
        print('The player is WIN')
        return True
    return False

def dealer_busts(player,dealer):
    if dealer.value > 21:
        player.chips.win_bet()
        print('The dealer is BUST. The player is WIN')
        return True
    return False
    
def dealer_wins(player,dealer):
    if dealer.value == 21:
        player.chips.lose_bet()
        print('The dealer is WIN')
        return True
    return False
    
def win_push(player,dealer):
    '''
    check hand player and dealer
    '''
    if player.value > dealer.value:
        player.chips.win_bet()
        print('The player is WIN')
    elif player.value < dealer.value:
        player.chips.lose_bet()
        print('The dealer is WIN') 
    else:
        print("Dealer and Player tie! It's a push.")
       

    
#Start game   

# Print an opening statement
print('Welcome to BlackJack! Get as close to 21 as you can without going over!\n\
    Dealer hits until she reaches 17. Aces count as 1 or 11.')
        
    
deck = Deck(4)
deck.shufle()

'''
init player hand and generate a start chips
'''
player = Hand()
player.chips = Chips()

while True:
     
    if deck.check_deck():
        print("The deck is almost empty. Thanks for playing!")
        break
    
    player.clear()
    player.add_card(deck.deal())
    player.add_card(deck.deal())
        
    dealer = Hand()
    dealer.add_card(deck.deal())
    dealer.add_card(deck.deal())

    take_bet(player.chips)
     
    show_some(player, dealer)

    if player_wins(player, dealer) or player_busts(player, dealer):
        break
            
    while playing: 
       
        hit_or_stand(deck, player)

        show_some(player, dealer)

        if player_wins(player, dealer) or player_busts(player, dealer):
            break
    
    if player.value < 21:
        
        show_all(player, dealer)
        
        if dealer_wins(player, dealer) or dealer_busts(player, dealer):
            break
        # If Player hasn't busted, play Dealer's hand until Dealer reaches 17
        while dealer.value < 18: 
            hit(deck, dealer)
            
        show_all(player, dealer)        

        if not(dealer_wins(player, dealer) or dealer_busts(player, dealer)):
            win_push(player,dealer) 
        
    # Inform Player of their chips total 
    print("\nPlayer's winnings stand at", player.chips.total)
        
    # Ask to play again    
    play_again = input('Do you want play again? Y or N: ')
    while play_again not in ['Y', 'N']:
        try:
            play_again = input('Do you want play again? Y or N: ').upper()
        except:
            print("Sorry, please try again")
            continue
    if play_again == 'N':
        print("Thanks for playing!")
        break
    else:
        playing = True
        continue

Welcome to BlackJack! Get as close to 21 as you can without going over!
    Dealer hits until she reaches 17. Aces count as 1 or 11.
Take a bet, please: 20
The bet is OK

Dealer's Hand:
 <card hidden>
 King of Spades

Player's Hand:
 Ten of Spades
 King of Spades
Player's Hand = 20
Would you like stand or hit? Enter s or hs
Player stands. Dealer is playing.

Dealer's Hand:
 <card hidden>
 King of Spades

Player's Hand:
 Ten of Spades
 King of Spades
Player's Hand = 20

Dealer's Hand:
 Nine of Hearts
 King of Spades
Dealer's Hand = 19

Player's Hand:
 Ten of Spades
 King of Spades
Player's Hand = 20

Dealer's Hand:
 Nine of Hearts
 King of Spades
Dealer's Hand = 19

Player's Hand:
 Ten of Spades
 King of Spades
Player's Hand = 20
The player is WIN

Player's winnings stand at 120
Do you want play again? Y or N: N
Thanks for playing!
